In [1]:
using Phylo
using BioSequences

In [2]:
f = open("dna.txt");
s = readlines(f)
close(f)


In [3]:
d = [split(x) for x in s if x != ""]
dna = Dict(x[1] => convert(DNASequence, x[2]) for x in d)

Dict{SubString{String},BioSequences.BioSequence{BioSequences.DNAAlphabet{4}}} with 11 entries:
  "U09126"   => GAGGGGATACAGAGGAATTGGAAACAATGGTGGATATGGGGCATC
  "U067158"  => AGGGGGACACTGAGGAATTATCAACAATGGTGGATATGGGGCGTC
  "L20571"   => AAAGTAATGAAGAAGAACAACAGGAAGTCATGGAGCTTATACATA
  "U27399"   => AGGGAGATGAGGAGGAATTGTCAGCATTTGTGGGGATGGGGCACC
  "AF025763" => AAGGGGATCAGGAAGAATTGTCAGCACTTGTGGAGATGGGGCATG
  "U43386"   => AGGGAGATGCAGAGGAATTATCAGCATTTATGGAAATGGGGCATC
  "L02317"   => AAGGAGATCAGGAAGAATTATCAGCACTTGTGGAGATGGGGCACC
  "U08443"   => AAGGAGATGAGGAAGCATTGTCAGCACTTATGGAGAGGGGGCACC
  "AF042106" => AAGGGGATCAGGAAGAATTATCGGCACTTGTGGACATGGGGCACC
  "U27445"   => AAGGGGATACGGACGAATTGGCAACACTTCTGGAGATGGGGAACT
  "U09127"   => ATGGGGATAGAGAGGAATTATCCTTGCTGGTGGACATGGGGGATT

In [4]:
tiplabels = convert(Array{String}, collect(keys(dna)))
treegen = Nonultrametric(tiplabels)

Phylo.Nonultrametric{Phylo.BinaryTree{Phylo.LeafInfo,Void},Distributions.Exponential}(11, String["U09126", "U067158", "L20571", "U27399", "AF025763", "U43386", "L02317", "U08443", "AF042106", "U27445", "U09127"], Distributions.Exponential{Float64}(θ=1.0))

In [5]:
tree = rand(treegen)

NamedTree phylogenetic tree with 21 nodes and 20 branches
Leaf names:
String["U09126", "U067158", "L20571", "U27399", "AF025763", "U43386", "L02317", "U08443", "AF042106", "U27445", "U09127"]

In [33]:
# Instantaneous rate matrix
# Jukes-Cantor model

function ratematrix(µ)
    Q = µ .* [-0.75 0.25 0.25 0.25; 0.25 -0.75 0.25 0.25; 0.25 0.25 -0.75 0.25; 0.25 0.25 0.25 -0.75]
    return(Hermitian(Q))
end

ratematrix (generic function with 1 method)

In [43]:
Q = ratematrix(0.9) # Instantaneous rate matrix
t = 0.2 # Time interval (units?)

@time expm(Q .* t)

  0.000240 seconds (53 allocations: 5.328 KiB)


4×4 Array{Float64,2}:
 0.876453   0.0411824  0.0411824  0.0411824
 0.0411824  0.876453   0.0411824  0.0411824
 0.0411824  0.0411824  0.876453   0.0411824
 0.0411824  0.0411824  0.0411824  0.876453 

In [44]:
nucleotides = convert(DNASequence, "ATCG")
base_to_idx = Dict(base => i for (i, base) in zip(1:4, nucleotides))


# Transition probability P
# ATCG
#P = expm(Q .* t)

#transition(Q, 0.2, DNA_G, DNA_C)

Dict{BioSymbols.DNA,Int64} with 4 entries:
  DNA_G => 4
  DNA_A => 1
  DNA_T => 2
  DNA_C => 3

In [45]:
#function transition(Q, t, 
#        base_in::BioSequences.BioSequence{BioSequences.DNAAlphabet{4}}, 
#        base_out::BioSequences.BioSequence{BioSequences.DNAAlphabet{4}})
#    Pm = expm(Q .* t)
#    
#    P = [Pm[base_to_idx[s], base_to_idx[x]] for (s, x) in zip(base_in, base_out)]
#    return(P)
#end


function transition(Q, 
        t, 
        base_in::BioSymbols.DNA, 
        base_out::BioSymbols.DNA)
    Pm = expm(Q .* t)
    
    #P = [Pm[base_to_idx[s], base_to_idx[x]] for (s, x) in zip(base_in, base_out)]
    P = Pm[base_to_idx[base_in], 
        base_to_idx[base_out]]
    return(P)
end

transition (generic function with 1 method)

In [46]:
typeof(DNA_C)

BioSymbols.DNA

In [89]:
bl = rand(1)
Q = ratematrix(rand(1))

println(bl)
@time transition(Q, 0.2, DNA_C, DNA_T)

[0.0756874]
  0.000122 seconds (33 allocations: 4.297 KiB)


0.007339697055200989

In [13]:
p_sum = 0
for x in keys(base_to_idx)
    p = transition(Q, 0.2, DNA_A, x)
    println(p)
    p_sum += p
end

p_sum

0.04118244714718208
0.8764526585584541
0.041182447147182055
0.041182447147182166


1.0000000000000004

In [90]:
n = length(collect(values(dna))[1])
alignment = Array{BioSymbols.DNA, 2}(length(dna), n)


for (row, sequence) in enumerate(values(dna))
    alignment[row,:] = [x for x in sequence]
end

freq = zeros(length(nucleotides), n)

for site_i in 1:size(alignment, 2)
    for (i, nucleotide) in enumerate(nucleotides)
        m = sum(x == nucleotide for x in alignment[:,site_i])/n
        freq[i, site_i] = m
    end
end

In [127]:
?Hermitian

search: Hermitian ishermitian



```
Hermitian(A, uplo=:U)
```

Construct a `Hermitian` view of the upper (if `uplo = :U`) or lower (if `uplo = :L`) triangle of the matrix `A`.

# Example

```jldoctest
julia> A = [1 0 2+2im 0 3-3im; 0 4 0 5 0; 6-6im 0 7 0 8+8im; 0 9 0 1 0; 2+2im 0 3-3im 0 4];

julia> Hupper = Hermitian(A)
5×5 Hermitian{Complex{Int64},Array{Complex{Int64},2}}:
 1+0im  0+0im  2+2im  0+0im  3-3im
 0+0im  4+0im  0+0im  5+0im  0+0im
 2-2im  0+0im  7+0im  0+0im  8+8im
 0+0im  5+0im  0+0im  1+0im  0+0im
 3+3im  0+0im  8-8im  0+0im  4+0im

julia> Hlower = Hermitian(A, :L)
5×5 Hermitian{Complex{Int64},Array{Complex{Int64},2}}:
 1+0im  0+0im  6+6im  0+0im  2-2im
 0+0im  4+0im  0+0im  9+0im  0+0im
 6-6im  0+0im  7+0im  0+0im  3+3im
 0+0im  9+0im  0+0im  1+0im  0+0im
 2+2im  0+0im  3-3im  0+0im  4+0im
```

Note that `Hupper` will not be equal to `Hlower` unless `A` is itself Hermitian (e.g. if `A == A'`).


In [125]:
sizeof(zeros(Float64, 4, 4))

128

In [126]:
#expm(Q)
sizeof(Q)

16

In [91]:
n = length(collect(values(dna))[1])
#DNA_alphabet = [convert(DNASequence, nucleotide ^ n) for nucleotide in ['A', 'T', 'C', 'G']]

function loglikelihood(tree, freq, Q)
    rootname = collect(keys(tree.nodes))[end]
    root = getnode(tree, rootname)
    branches = getbranches(tree)

    
    result = zeros(Float64, length(nucleotides), n)

    for (i, x) in enumerate(nucleotides)
        #println(x)
        result[i,:] = L(x, root, tree, Q, branches)
    end

    logL = sum(log.(sum(result .* freq, 1)))
    return(logL)
end

## Felsenstein pruning algorithm
## Recursive, start at putative "root". For time-reversible models, position
## of root does not matter, as L is the same

## Vectorized for DNA sequence of length >= 1
function L(s, node, tree, Q, branches)
    if isleaf(node)
        nodename = branches[node.inbound].destination
        tip_sequence = dna[nodename]

        res = [s == x for x in tip_sequence]
        return(res)
    else
        res = zeros(Float64, n, length(node.outbounds))

        for (i, node_idx) in enumerate(node.outbounds)
            branch = branches[node_idx]
            node = getnode(tree, branch.destination)

            P = zeros(Float64, n, 4)
            for (ii, x) in enumerate(nucleotides)
                p = transition(Q, branch.length, s, x)
                l = L(x, node, tree, Q, branches)

                P[:,ii] = p .* l
                
            end
            res[:,i] = reshape(sum(P, 2), n)
        end
        #println(res)
        return(prod(res, 2))
    end
end


L (generic function with 1 method)

In [95]:
res = @time @profile loglikelihood(tree, freq, Q)
#res = @time foo()

  0.772462 seconds (1.35 M allocations: 110.846 MiB, 2.38% gc time)


-452.6529749951561

In [96]:
#Profile.print()

In [104]:
trees = [rand(treegen) for x in 1:10];

10-element Array{Phylo.BinaryTree{Phylo.LeafInfo,Void},1}:
 NamedTree phylogenetic tree with 21 nodes (11 leaves) and 20 branches
 NamedTree phylogenetic tree with 21 nodes (11 leaves) and 20 branches
 NamedTree phylogenetic tree with 21 nodes (11 leaves) and 20 branches
 NamedTree phylogenetic tree with 21 nodes (11 leaves) and 20 branches
 NamedTree phylogenetic tree with 21 nodes (11 leaves) and 20 branches
 NamedTree phylogenetic tree with 21 nodes (11 leaves) and 20 branches
 NamedTree phylogenetic tree with 21 nodes (11 leaves) and 20 branches
 NamedTree phylogenetic tree with 21 nodes (11 leaves) and 20 branches
 NamedTree phylogenetic tree with 21 nodes (11 leaves) and 20 branches
 NamedTree phylogenetic tree with 21 nodes (11 leaves) and 20 branches

In [121]:
using ProgressMeter
res = []

@time @showprogress 1 for tree in trees
    logL = loglikelihood(tree, freq, Q)
    #logL = foo()
    append!(res, logL)
end

Progress:  90%|█████████████████████████████████████    |  ETA: 0:00:02

 27.886408 seconds (56.36 M allocations: 4.535 GiB, 2.72% gc time)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:28


In [120]:
tr = rand(treegen)
@time loglikelihood(tr, freq, Q)

  8.851803 seconds (17.97 M allocations: 1.446 GiB, 2.71% gc time)


-469.2890942083639

In [643]:
for x in [DNA_alphabet[1], tree, Q, freq, node]
    println(typeof(x))
end

BioSequences.BioSequence{BioSequences.DNAAlphabet{4}}
Phylo.BinaryTree{Phylo.LeafInfo,Void}
Array{Float64,2}
Array{Float64,2}
Phylo.BinaryNode{Int64}


In [112]:
sizeof(tree)

40

In [114]:
typeof(tree)

Phylo.BinaryTree{Phylo.LeafInfo,Void}

In [115]:
sizeof(Phylo.BinaryTree{Phylo.LeafInfo,Void})

40